# Profiling python code

In [1]:
import numpy as np

In [2]:
n = 1000000
a = np.random.randn(n)
b = np.random.randn(n)
c = np.empty(n, dtype='float64')

In [3]:
%%time
c = a + b

CPU times: user 3.22 ms, sys: 67 μs, total: 3.29 ms
Wall time: 4.44 ms
